In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical, Normal
import pandas as pd
import numpy as np
# Constants
num_systems = 4
num_continuous_controls = num_systems * 9 #冷机冷冻水出水温度，冷却水进水温度,冷塔频率1,2,3,4，冷却塔冷却水出水温度,冷却泵功率,冷冻泵功率

num_features = 29*4


num_towers_per_system=4
# For each system:
num_chiller_actions = 2  # On or off
num_plate_exchanger_actions = 2  # On or off
num_cooling_tower_combinations = 2 * num_towers_per_system  # Each can be on or off

# The action to turn the entire system on or off (master switch)
num_system_off_action = 2

# For num_systems systems, the discrete action space is:
num_discrete_controls_per_system = num_chiller_actions + num_plate_exchanger_actions + num_cooling_tower_combinations
num_discrete_controls = ((num_discrete_controls_per_system+num_system_off_action) * num_systems) 

print('num_continuous_controls: {} , num_discrete_controls: {}'.format(num_continuous_controls,num_discrete_controls))

max_time=None

# Hyperparameters
learning_rate = 0.01
gamma = 0.99  # Discount factor for rewards

# Policy network
class PolicyNetwork(nn.Module):
    def __init__(self, num_features, num_continuous_controls, num_discrete_controls):
        super(PolicyNetwork, self).__init__()
        # Shared layers
        self.shared_layers = nn.Sequential(
            nn.Linear(num_features, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU()
        )
        
         # Branch for discrete controls
        self.discrete_head = nn.Sequential(
            nn.Linear(64, num_discrete_controls)  # Probabilities for discrete actions
        )
        
        
        # Branch for continuous controls
        self.continuous_head = nn.Sequential(
            nn.Linear(64, num_continuous_controls * 2)  # Mean and std dev for each control
        )
        
       
    def forward(self, x):
        x = self.shared_layers(x)
    
        # Discrete actions output
        discrete_logits = self.discrete_head(x)
        
        # Continuous actions output
        continuous_actions = self.continuous_head(x)
        means = continuous_actions[:, :num_continuous_controls]
        std_devs = torch.clamp(continuous_actions[:, num_continuous_controls:], min=1e-3)
        
        return means, std_devs, discrete_logits

# Instantiate the policy network and the optimizer
policy = PolicyNetwork(num_features, num_continuous_controls, num_discrete_controls)
optimizer = optim.Adam(policy.parameters(), lr=learning_rate)



def discount_rewards(rewards, gamma):
    discounted = []
    cumulative_total = 0
    # 从后向前计算折扣后的奖励总和
    for reward in rewards[::-1]:
        cumulative_total = reward + cumulative_total * gamma
        discounted.insert(0, cumulative_total)
    return discounted

#分割冷却塔功率
def process_and_sum_indices(current_state, indices):
        total_sum = 0
        for index in indices:
            if index < len(current_state):
                # Retrieve the value at the specified index
                value = current_state[index]

                # Check if the value is a string that needs to be split and summed
                if isinstance(value, str) and '/' in value:
                    # Split the string into parts based on '/'
                    parts = value.split('/')
                    # Convert each part to float and add to the total sum
                    total_sum += sum(float(part) for part in parts)
                else:
                    # For normal numeric values, add directly to the total sum
                    total_sum += float(value)
            else:
                print(f"Warning: Index {index} is out of range.")
    
        return total_sum

#计算 reward
def calculate_reward_from_power_consumption(current_state, next_state):
    

    # List of indices we want to sum up, including index 18 with the complex string
    indices = [12, 14, 16, 18]

    # Calculate the sum of the values at the specified indices
    total_old_power = process_and_sum_indices(current_state, indices)

    total_new_power = process_and_sum_indices(next_state,indices)

    # 奖励为功率减少的量，如果功率增加，则奖励为负
    reward = total_old_power - total_new_power
    return reward

#给current_state normalize feature
def normalize(data):
    """
    Normalize a list of features with zero-padding handling.

    Parameters:
        data (list): The list of features to normalize, possibly zero-padded.
        feature_mins (list): The minimum values for each feature across all data.
        feature_maxs (list): The maximum values for each feature across all data.
    
    Returns:
        list: The normalized list of features.
    """
    feature_mins = [0, 10, 20, 5, 0, 10, 20, 5, 0, 10, 0, 50, 0, 20, 0, 20, 0, 10,10,10,10,10,10,10,10,10,10,10,10] * 4  # Repeat pattern for all systems
    feature_maxs = [100, 50, 80, 60, 100, 50, 80, 60, 100, 50, 100, 300, 100, 100, 100, 100, 100, 50,30,30,30,30,30,30,30,30,30,30,30] * 4
    normalized_data = []
    for i, value in enumerate(data):
        value=float(value)
        if value == 0:  # Assuming 0 is only used for zero-padding
            normalized_data.append(0)
        else:
            # Apply Min-Max normalization
            min_val = feature_mins[i]
            max_val = feature_maxs[i]
            normalized_val = (value - min_val) / (max_val - min_val) if max_val > min_val else 0
            normalized_data.append(normalized_val)
    
    return normalized_data

def strlist2float(stringlist):
    float_list = []
    for item in stringlist:
        try:
            float_item = float(item)
        except ValueError:
            print(f"Warning: '{item}' is not a valid float. Using default value {default_value}.")
            float_item = default_value  # Use the default value
        float_list.append(float_item)
    return float_list
        
#拿到一个系统的数据(helper function)
def get_system_data(df,system_id):
    try:
        # This function would fetch or receive the current operational data for a given system
        columns=['冷机冷冻回水温度', '冷机冷冻出水温度',
           '板换冷冻回水温度', '板换冷冻出水温度', '冷机冷却回水温度', '冷机冷却出水温度', '板换冷却回水温度', '板换冷却出水温度',
           '冷塔出水温度', '冷塔回水温度', '冷机负载率', '冷机功率', '冷冻水泵频率', '冷冻水泵功率', '冷却水泵频率',
           '冷却水泵功率', '冷塔频率', '冷塔功率', '冷冻水流量', '冷却水流量','室外干球', '室外湿度', '室外湿球']

        # Construct the system identifier string
        target_identifier = f"{system_id}#系统"

        # Find the row with the matching 'System Identifier'
        row = df[df['System Identifier'] == target_identifier]
       
        if not row.empty:
            data = row[columns].iloc[0].to_list()

            # Define the required number of sub-items for indices 17 and 18
            required_count = 4

            # Process the list to handle '/' separated values and validate specific indices
            processed_data = []
            split_values_16 = None
            split_values_17 = None

            for index, item in enumerate(data):
                if isinstance(item, str) and '/' in item:
                    sub_items = item.split('/')

                    # If indices 17 or 18, check number of sub-items and adjust
                    if index == 16 or index == 17:
                        if len(sub_items) < required_count:
                            # Add zeros if there are not enough sub-items
                            sub_items.extend(['0'] * (required_count - len(sub_items)))
                        elif len(sub_items) > required_count:
                            # Raise an exception if there are more sub-items than expected
                            raise ValueError(f"Index {index} has more sub-values than expected ({required_count}).")

                        # Store the results to compare later
                        if index == 16:
                            split_values_16 = sub_items
                        else:
                            split_values_17 = sub_items

                    # Append or extend the processed data list
                    
                    processed_data.extend(strlist2float(sub_items))
                else:
                    if np.isnan(item):
                        item=float(0)
                    # Append non-split items directly
                    processed_data.append(float(item))

            # After both are processed, ensure they have the same number of items
            if split_values_16 and split_values_17 and len(split_values_16) != len(split_values_17):
                raise ValueError("The number of items at index 17 and 18 do not match after processing.")
            return processed_data
        else:
            # Return None if no matching row is found
            return None
    except ValueError as e:
        print("Error:", e)
        return None
    
    
#     return [chiller_power, pump_power, cooling_water_power, tower_power]

def is_system_running(latest_df,system_id):
    
    identifiers = []

    # Iterating through the rows and checking the '运行模式'
    for index, row in latest_df.iterrows():
        if row['运行模式'] is not None:  # Check if '运行模式' is not None
            identifiers.append(row['System Identifier'])  # Add the 'System Identifier' to the list
    
    target_identifier = f"{system_id}#系统"
    
    # Check if the target identifier exists in the 'System Identifier' column
    return target_identifier in latest_df['System Identifier'].values

#拿到系统现在状态
def get_real_system_state():
    df = pd.read_excel("/Users/zhiranbai/Downloads/工作/数据中心AI/2冷源流量补值代码-3种模式/补充混合+板换+冷机水流量.xlsx")
    # columns_to_convert = ['冷机功率','冷冻水泵功率','冷却水泵功率','冷塔功率']
    # for col in columns_to_convert:
    #     df[col] = pd.to_numeric(df[col], errors='coerce')
    global max_time
    max_time = df['MM-DD hour'].max()

    latest_time_rows = df[df['MM-DD hour'] == max_time]    
        
    current_state = []
    for system_id in range(1, num_systems + 1):
        if is_system_running(latest_time_rows,system_id):
            
            
            
            system_data = None
            while system_data is None:
                system_data = get_system_data(latest_time_rows,system_id)
                current_state.extend(system_data)
                print("system_data: {}" .format(system_data))
                if system_data is None:
                    input("Please correct the data and press Enter to retry...")
            print("Data processed successfully:", system_data)

            
            
            
        else:
            system_data = [0] * 29
            current_state.extend(system_data)

    current_state = np.array(normalize(current_state)) # Apply normalization or standardization
    
    
    return current_state


def scale_continuous_actions(actions):
    
    min_values=[1,2,3,4,5,6,7,8,9]*4  #找人
    max_values=[11,12,13,14,15,16,17,18,19]*4 #找人
    continuous_name=['系统1:冷机冷冻水出水温度','系统1:冷却水进水温度','系统1:冷塔频率1','系统1:冷塔频率2','系统1:冷塔频率3','系统1:冷塔频率4','系统1:冷却塔冷却水出水温度','系统1:冷却泵功率','系统1:冷冻泵功率','系统2:冷机冷冻水出水温度','系统2:冷却水进水温度','系统2:冷塔频率1','系统2:冷塔频率2','系统2:冷塔频率3','系统2:冷塔频率4','系统2:冷却塔冷却水出水温度','系统2:冷却泵功率','系统2:冷冻泵功率','系统3:冷机冷冻水出水温度','系统3:冷却水进水温度','系统3:冷塔频率1','系统3:冷塔频率2','系统3:冷塔频率3','系统3:冷塔频率4','系统3:冷却塔冷却水出水温度','系统3:冷却泵功率','系统3:冷冻泵功率','系统4:冷机冷冻水出水温度','系统4:冷却水进水温度','系统4:冷塔频率1','系统4:冷塔频率2','系统4:冷塔频率3','系统4:冷塔频率4','系统4:冷却塔冷却水出水温度','系统4:冷却泵功率','系统4:冷冻泵功率']
    scaled_actions = []
    if isinstance(actions, torch.Tensor):
        actions = actions.squeeze()
    for action, min_value, max_value, action_name in zip(actions, min_values, max_values,continuous_name):
        scaled_action = action.item() * (max_value - min_value) + min_value
        scaled_actions.append((action_name,scaled_action))
        
        
    return scaled_actions

def apply_discrete_action(action):
    """ Apply a discrete action based on the output probabilities. """
   
    category_List = ['System 1 Chiller: OFF', 'System 1 Chiller: ON', 'System 1 Plate Exchanger: OFF', 'System 1 Plate Exchanger: ON', 'System 1 Cooling Tower 1: OFF', 'System 1 Cooling Tower 1: ON', 'System 1 Cooling Tower 2: OFF', 'System 1 Cooling Tower 2: ON', 'System 1 Cooling Tower 3: OFF', 'System 1 Cooling Tower 3: ON', 'System 1 Cooling Tower 4: OFF', 'System 1 Cooling Tower 4: ON', 'System 1 whole: OFF','System 1 whole: ON','System 2 Chiller: OFF', 'System 2 Chiller: ON', 'System 2 Plate Exchanger: OFF', 'System 2 Plate Exchanger: ON', 'System 2 Cooling Tower 1: OFF', 'System 2 Cooling Tower 1: ON', 'System 2 Cooling Tower 2: OFF', 'System 2 Cooling Tower 2: ON', 'System 2 Cooling Tower 3: OFF', 'System 2 Cooling Tower 3: ON', 'System 2 Cooling Tower 4: OFF', 'System 2 Cooling Tower 4: ON','System 2 whole: OFF','System 2 whole: ON','System 3 Chiller: OFF', 'System 3 Chiller: ON', 'System 3 Plate Exchanger: OFF', 'System 3 Plate Exchanger: ON', 'System 3 Cooling Tower 1: OFF', 'System 3 Cooling Tower 1: ON', 'System 3 Cooling Tower 2: OFF', 'System 3 Cooling Tower 2: ON', 'System 3 Cooling Tower 3: OFF', 'System 3 Cooling Tower 3: ON', 'System 3 Cooling Tower 4: OFF', 'System 3 Cooling Tower 4: ON','System 3 whole: OFF','System 3 whole: ON','System 4 Chiller: OFF', 'System 4 Chiller: ON', 'System 4 Plate Exchanger: OFF', 'System 4 Plate Exchanger: ON', 'System 4 Cooling Tower 1: OFF', 'System 4 Cooling Tower 1: ON', 'System 4 Cooling Tower 2: OFF', 'System 4 Cooling Tower 2: ON', 'System 4 Cooling Tower 3: OFF', 'System 4 Cooling Tower 3: ON', 'System 4 Cooling Tower 4: OFF', 'System 4 Cooling Tower 4: ON', 'System 4 whole: OFF','System 4 whole: ON'] 
   
    chosen_action= category_List[action]
    return chosen_action
# def control_loop(policy_network, system_state):
#     # Assume system_state is input to your network and it outputs action probabilities
#     state_tensor = torch.from_numpy(system_state).float().unsqueeze(0)
#     means, std_devs, discrete_logits = policy_network(state_tensor)

#     # Apply continuous actions
#     temp_control = scale_continuous_action(means[0].item(), 20, 35)
#     send_command_to_system(temp_control, system_id=1, parameter="temperature")

#     # Apply discrete actions
#     chiller_state = apply_discrete_action(discrete_logits[0])
#     send_command_to_system(chiller_state, system_id=1, parameter="chiller_state")


    
    
# This control loop would be called periodically or in response to system state changes

# Training function for policy gradients
def train_policy_gradient(policy, optimizer, states, actions, rewards):
    discounted_rewards = discount_rewards(rewards, gamma)
    policy_loss = []
    for state, action, reward in zip(states, actions, discounted_rewards):
        state = torch.tensor(state, dtype=torch.float32)
        action_probs = policy(state)
        # We will need to expand this for the multi-action space
        action_distribution = Categorical(action_probs)
        loss = -action_distribution.log_prob(action) * reward
        policy_loss.append(loss)

    optimizer.zero_grad()
    policy_loss = torch.stack(policy_loss).sum()
    policy_loss.backward()
    optimizer.step()
    


    
    
def apply_actions_to_real_system(continuous_actions, discrete_actions):
    global max_time
    print(continuous_actions, discrete_actions)
    df = pd.read_excel("/Users/zhiranbai/Downloads/工作/数据中心AI/2冷源流量补值代码-3种模式/补充混合+板换+冷机水流量.xlsx")
    new_max_time = df['MM-DD hour'].max()
    while True:  # This creates an infinite loop
        user_input = input("当操作执行完成，输入 'done': ")  # Prompt for input
        if user_input.lower() == 'done'and new_max_time != max_time:  # Check if the input is 'done' (case-insensitive)
            print("确认, 获取数据中...")
            break  # Exit the loop if the condition is met
        else:
            print("等待确认...")
            

        next_latest_time_rows = df[df['MM-DD hour'] == new_max_time]    

        next_state = []
        for system_id in range(1, num_systems + 1):
            if is_system_running(next_latest_time_rows,system_id):
                system_data = get_system_data(latest_time_rows,system_id)
            else:
                system_data = [0] * 23
                next_state.extend(system_data)

        next_state = normalize(next_state)  # Apply normalization or standardization


        return next_state
    

    

# Training loop
num_episodes = 1000
    
for episode in range(num_episodes):
    # Retrieve the current state of the real environment
    current_state = get_real_system_state()  # This function needs to be implemented to fetch real-world data
   
    # Initialize episode memory
    states, actions, rewards = [], [], []
    done = False
    
    while not done:
        state_tensor = torch.from_numpy(current_state).float().unsqueeze(0)
       
        means, std_devs, discrete_logits = policy(state_tensor)

        # Sample from the distributions for continuous actions
        continuous_distribution = Normal(means, std_devs)
        continuous_actions = continuous_distribution.sample()
        continuous_actions=scale_continuous_actions(continuous_actions)
        # Apply constraints to continuous actions here if necessary
        # ...

        # Sample from the distribution for discrete actions
        discrete_distribution = Categorical(logits=discrete_logits)
        discrete_action_index = discrete_distribution.sample()
        discrete_action = apply_discrete_action(discrete_action_index)
        # Combine and possibly constrain the discrete actions here
        # ...

        # Apply the actions to the real system and get the new state and reward
        # Ensure this is done in a safe manner with proper error checking
        next_state= apply_actions_to_real_system(continuous_actions, discrete_action)
        
        # ...

        states.append(current_state)
        actions.append((continuous_actions, discrete_action))  # Store actions taken

#         Calculate reward based on the power consumption difference
        reward = calculate_reward_from_power_consumption(current_state, next_state)
#         ...

        rewards.append(reward)
        current_state = next_state

        # Implement your stopping criteria
        if check_episode_done(new_state):
            done=True
        # ...

    # Update policy after each episode or after collecting enough data
    train_policy_gradient(policy, optimizer, states, actions, rewards)

num_continuous_controls: 36 , num_discrete_controls: 56
system_data: [20.1, 16.5, 0.0, 0.0, 19.2, 22.7, 0.0, 0.0, 19.7, 26.3, 0.0, 280.0, 40.0, 77.2, 48.6, 96.9, 50.0, 50.0, 50.0, 0.0, 81.3, 81.3, 81.3, 0.0, 751.0, 635.0, 15.8, 67.4, 12.4]
Data processed successfully: [20.1, 16.5, 0.0, 0.0, 19.2, 22.7, 0.0, 0.0, 19.7, 26.3, 0.0, 280.0, 40.0, 77.2, 48.6, 96.9, 50.0, 50.0, 50.0, 0.0, 81.3, 81.3, 81.3, 0.0, 751.0, 635.0, 15.8, 67.4, 12.4]
system_data: [18.7, 14.9, 0.0, 0.0, 17.2, 22.0, 0.0, 0.0, 19.6, 21.3, 0.0, 389.0, 39.3, 81.1, 48.8, 110.0, 34.1, 34.1, 34.1, 34.1, 8.8, 8.9, 8.9, 9.1, 646.0, 733.0, 15.8, 67.4, 12.4]
Data processed successfully: [18.7, 14.9, 0.0, 0.0, 17.2, 22.0, 0.0, 0.0, 19.6, 21.3, 0.0, 389.0, 39.3, 81.1, 48.8, 110.0, 34.1, 34.1, 34.1, 34.1, 8.8, 8.9, 8.9, 9.1, 646.0, 733.0, 15.8, 67.4, 12.4]
[('系统1:冷机冷冻水出水温度', 11.184035301208496), ('系统1:冷却水进水温度', 16.87858772277832), ('系统1:冷塔频率1', 1.456447422504425), ('系统1:冷塔频率2', -21.749950408935547), ('系统1:冷塔频率3', 8.00382375717163),

In [34]:
df = pd.read_excel("/Users/zhiranbai/Downloads/工作/数据中心AI/2冷源流量补值代码-3种模式/补充混合+板换+冷机水流量.xlsx")
# columns_to_convert = ['冷机功率','冷冻水泵功率','冷却水泵功率','冷塔功率']
# for col in columns_to_convert:
#     df[col] = pd.to_numeric(df[col], errors='coerce')
max_time = df['MM-DD hour'].max()

latest_time_rows = df[df['MM-DD hour'] == max_time]
identifiers = []
def get_system_data(df,system_id):
    # This function would fetch or receive the current operational data for a given system
    columns=['运行模式', '冷机冷冻回水温度', '冷机冷冻出水温度',
       '板换冷冻回水温度', '板换冷冻出水温度', '冷机冷却回水温度', '冷机冷却出水温度', '板换冷却回水温度', '板换冷却出水温度',
       '冷塔出水温度', '冷塔回水温度', '冷机负载率', '冷机功率', '冷冻水泵频率', '冷冻水泵功率', '冷却水泵频率',
       '冷却水泵功率', '冷塔频率', '冷塔功率', '冷冻水流量', '冷却水流量','室外干球', '室外湿度', '室外湿球']
    
    # Construct the system identifier string
    target_identifier = f"{system_id}#系统"
    
    # Find the row with the matching 'System Identifier'
    row = df[df['System Identifier'] == target_identifier]
    
    if not row.empty:
        cols=row[columns].iloc[0].to_list()
        cols[17], cols[18],
        return row[columns].iloc[0].to_list()
    else:
        # Return None if no matching row is found
        return None
# Iterating through the rows and checking the '运行模式'
for index, row in latest_time_rows.iterrows():
    if row['运行模式'] is not None:  # Check if '运行模式' is not None
        identifiers.append(row['System Identifier'])  # Add the 'System Identifier' to the list

print(identifiers)

print(get_system_data(latest_time_rows,1))
      
print(latest_time_rows.shape[0]) 
print(latest_time_rows.columns)        
#System Identifier	运行模式	冷机冷冻回水温度	冷机冷冻出水温度	板换冷冻回水温度	板换冷冻出水温度	冷机冷却回水温度	冷机冷却出水温度	板换冷却回水温度	板换冷却出水温度	冷塔出水温度	冷塔回水温度	冷机负载率	冷机功率	冷冻水泵频率	冷冻水泵功率	冷却水泵频率	冷却水泵功率	冷塔频率	冷塔功率	冷冻水流量	冷却水流量	冷机电流百分比	集水器1回水温度	集水器2回水温度	分水器1供水温度	分水器2供水温度	压差	室外干球	室外湿度	室外湿球

['1#系统', '4#系统']
['冷机模式', 20.1, 16.5, nan, nan, 19.2, 22.7, nan, nan, 19.7, 26.3, nan, 280.0, 40.0, 77.2, 48.6, 96.9, '50/50/50', '81.3', 751.0, 635.0, 15.8, 67.4, 12.4]
2
Index(['MM-DD hour', 'System Identifier', '运行模式', '冷机冷冻回水温度', '冷机冷冻出水温度',
       '板换冷冻回水温度', '板换冷冻出水温度', '冷机冷却回水温度', '冷机冷却出水温度', '板换冷却回水温度', '板换冷却出水温度',
       '冷塔出水温度', '冷塔回水温度', '冷机负载率', '冷机功率', '冷冻水泵频率', '冷冻水泵功率', '冷却水泵频率',
       '冷却水泵功率', '冷塔频率', '冷塔功率', '冷冻水流量', '冷却水流量', '冷机电流百分比', '集水器1回水温度',
       '集水器2回水温度', '分水器1供水温度', '分水器2供水温度', '压差', '室外干球', '室外湿度', '室外湿球'],
      dtype='object')


In [35]:
latest_time_rows

,MM-DD hour,System Identifier,运行模式,冷机冷冻回水温度,冷机冷冻出水温度,板换冷冻回水温度,板换冷冻出水温度,冷机冷却回水温度,冷机冷却出水温度,板换冷却回水温度,...,冷却水流量,冷机电流百分比,集水器1回水温度,集水器2回水温度,分水器1供水温度,分水器2供水温度,压差,室外干球,室外湿度,室外湿球
10073,2024-04-20,1#系统,冷机模式,20.1,16.5,NaN,NaN,19.2,22.7,NaN,...,635.0,42.7,NaN,NaN,NaN,NaN,NaN,15.8,67.4,12.4
10074,2024-04-20,4#系统,冷机模式,18.7,14.9,NaN,NaN,17.2,22.0,NaN,...,733.0,62.7,21.0,19.4,16.0,15.9,103.0,15.8,67.4,12.4


In [3]:
num_systems = 4
num_towers_per_system = 4

actions = []

for system_id in range(1, num_systems + 1):
    # Chiller actions for each system
    actions.append(f"System {system_id} Chiller: OFF")
    actions.append(f"System {system_id} Chiller: ON")

    # Plate Exchanger actions for each system
    actions.append(f"System {system_id} Plate Exchanger: OFF")
    actions.append(f"System {system_id} Plate Exchanger: ON")

    # Cooling tower actions for each system
    for tower_id in range(1, num_towers_per_system + 1):
        actions.append(f"System {system_id} Cooling Tower {tower_id}: OFF")
        actions.append(f"System {system_id} Cooling Tower {tower_id}: ON")

# Master switch actions for the entire set of systems
actions.append("Master System: OFF")
actions.append("Master System: ON")

print(actions)

['System 1 Chiller: OFF', 'System 1 Chiller: ON', 'System 1 Plate Exchanger: OFF', 'System 1 Plate Exchanger: ON', 'System 1 Cooling Tower 1: OFF', 'System 1 Cooling Tower 1: ON', 'System 1 Cooling Tower 2: OFF', 'System 1 Cooling Tower 2: ON', 'System 1 Cooling Tower 3: OFF', 'System 1 Cooling Tower 3: ON', 'System 1 Cooling Tower 4: OFF', 'System 1 Cooling Tower 4: ON', 'System 2 Chiller: OFF', 'System 2 Chiller: ON', 'System 2 Plate Exchanger: OFF', 'System 2 Plate Exchanger: ON', 'System 2 Cooling Tower 1: OFF', 'System 2 Cooling Tower 1: ON', 'System 2 Cooling Tower 2: OFF', 'System 2 Cooling Tower 2: ON', 'System 2 Cooling Tower 3: OFF', 'System 2 Cooling Tower 3: ON', 'System 2 Cooling Tower 4: OFF', 'System 2 Cooling Tower 4: ON', 'System 3 Chiller: OFF', 'System 3 Chiller: ON', 'System 3 Plate Exchanger: OFF', 'System 3 Plate Exchanger: ON', 'System 3 Cooling Tower 1: OFF', 'System 3 Cooling Tower 1: ON', 'System 3 Cooling Tower 2: OFF', 'System 3 Cooling Tower 2: ON', 'System